In [4]:
import spacy 
import pandas as pd
import datetime as dt 
import spacy
from collections import Counter 
import json
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

In [5]:
submissions1 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions1.json")
submissions2 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions2.json")
submissions3 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions3.json")
submissions4 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions4.json")
submissions5 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions5.json")
submissions6 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions6.json")
submissions7 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions7.json")


### This analysis aims to track the frequencies of certain words over time

In [22]:
submissions = pd.concat([submissions1, submissions2, submissions3, submissions4, submissions5, submissions6, submissions7])

In [23]:
#create a column with real datetimes
submissions['created_utc'] = submissions['created_utc'].apply(lambda date: dt.datetime.fromtimestamp(date))
submissions = submissions.sort_values(by='created_utc', ascending=True)

In [24]:
#remove the stop words and punctionationfrom a line of text
def remove_stops(text):
    nlp = spacy.load("en_core_web_sm")
    words = [word.lower() for word in text.split() if word.lower() not in sw_spacy]
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    new_text = " ".join(words)
    for elem in punc:
        new_text = new_text.replace(elem, "")
    return new_text

In [31]:
posts = submissions[submissions['created_utc'].dt.year==2014]
curr_df = {"january": posts[posts['created_utc'].dt.month==1],
           "febuary": posts[posts['created_utc'].dt.month==2],
           "march": posts[posts['created_utc'].dt.month==3],
            "april": posts[posts['created_utc'].dt.month==4],
            "may": posts[posts['created_utc'].dt.month==5],
            "june": posts[posts['created_utc'].dt.month==6],
            "august": posts[posts['created_utc'].dt.month==8],
            "september": posts[posts['created_utc'].dt.month==9],
            "october": posts[posts['created_utc'].dt.month==10],
            "november": posts[posts['created_utc'].dt.month==11],
            "december": posts[posts['created_utc'].dt.month==12]}

In [32]:
#clean the text by removing stop words, punctuation, and empty entries
for month in curr_df.keys():
    curr_month = curr_df[month]
    text = curr_month['selftext'].to_list()
    text = [remove_stops(t) for t in text]
    while("" in text):
        text.remove("")
    text = " ".join(text)
    text = text.split()
    frequent = Counter(text) 
    most_occur = frequent.most_common(50) 
    json_format = json.dumps(dict(most_occur))
    with open(month + ".json", "w") as outfile:
        outfile.write(json_format)